# Train Custom Densenet-121 Model

## Import custom densenet-121 model

In [1]:
import sys
# import the py file for loading the dataset
if "..\\121-layer\\src" not in sys.path:
    sys.path.insert(0,r'..\121-layer\src')
print(sys.path)

['..\\121-layer\\src', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\notebooks', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv', '', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages', 'C:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\Pythonwin']


In [2]:
import torch
torch.manual_seed(42)
import random
random.seed(42)
import numpy as np
np.random.seed(42)

from custom_densenet import *
from preprocessing import *
from train_densenet import *
train_dataset, val_dataset,train_loader, val_loader,test_dataset, test_loader= get_data_loaders(data_dir='../raw_data/archive/', label_file='../raw_data/archive/CXR8-selected/Data_Entry_2017_v2020.csv')

Training Dataset Size: 8484
Validation Dataset Size: 1060
Test Dataset Size: 1060


## Train model

In [3]:
import torch
import torch.nn as nn

In [4]:
num_class = 2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

## custom implementation

model = dense_net(num_class-1, training = True) #binary so only need one output
model.to(device)

## official implementation
# import torchvision.models.densenet
# import torchvision

# model = torchvision.models.densenet121(weights='DenseNet121_Weights.DEFAULT')
# kernelCount = model.classifier.in_features
# model.classifier = nn.Sequential(nn.Linear(kernelCount, num_class -1), nn.Sigmoid())
# model.to(device)

dense_net(
  (initial_setup): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (denseblock1): dense_block(
    (dense_layer0): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
        (5): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (dense_layer1): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [5]:
train(model = model,
      train_loader = train_loader,
      train_dataset_length = len(train_dataset),
      val_loader = val_loader,
      num_class = num_class,
      device = device,
      model_name = "custom_dnet_binary_by_img_count_lr_1e-4_long",
      lr = 1e-4
    )

Epoch [1/50], Step [1/531], Loss: 0.6789, tp_sum: 4.0000, fp_sum: 1.0000, fn_sum: 4.0000, batch_f1_score: 0.6154
Epoch [1/50], Step [21/531], Loss: 0.6233, tp_sum: 5.0000, fp_sum: 5.0000, fn_sum: 1.0000, batch_f1_score: 0.6250
Epoch [1/50], Step [41/531], Loss: 0.7339, tp_sum: 5.0000, fp_sum: 3.0000, fn_sum: 6.0000, batch_f1_score: 0.5263
Epoch [1/50], Step [61/531], Loss: 0.6329, tp_sum: 5.0000, fp_sum: 2.0000, fn_sum: 2.0000, batch_f1_score: 0.7143
Epoch [1/50], Step [81/531], Loss: 0.7525, tp_sum: 4.0000, fp_sum: 6.0000, fn_sum: 3.0000, batch_f1_score: 0.4706
Epoch [1/50], Step [101/531], Loss: 0.7139, tp_sum: 6.0000, fp_sum: 4.0000, fn_sum: 4.0000, batch_f1_score: 0.6000
Epoch [1/50], Step [121/531], Loss: 0.6963, tp_sum: 8.0000, fp_sum: 2.0000, fn_sum: 5.0000, batch_f1_score: 0.6957
Epoch [1/50], Step [141/531], Loss: 0.6259, tp_sum: 4.0000, fp_sum: 4.0000, fn_sum: 3.0000, batch_f1_score: 0.5333
Epoch [1/50], Step [161/531], Loss: 0.7049, tp_sum: 5.0000, fp_sum: 2.0000, fn_sum: 5.